## Significance testing for best Model Results

Best models:

* Manual features: Retrained Random Forest classifier
* Visual features: ResNet50 freeze CNN classifier
* Combined features: VGG16 freeze all level features features + manual features Random Forest classifier


In [32]:
import pandas as pd
import numpy as np
import sklearn


In [33]:
# load predictions with baseline model (RF on manual features)
baseline_pred = pd.read_pickle(r'E:\Babette\MasterThesis\McNemarTest\baseline_rf.pkl')
resnet_ft_pred = pd.read_pickle(r'E:\Babette\MasterThesis\McNemarTest\ResNet_finetuned.pkl')
vis_man_pred = pd.read_pickle((r'E:\Babette\MasterThesis\McNemarTest\man_vis_rf.pkl'))

In [34]:
resnet_ft_pred= resnet_ft_pred.sort_values(by=['id']).reset_index()
baseline_pred['y_pred']= baseline_pred['y_pred'].astype(int)

In [35]:
vis_man_pred

,id,y_pred,y_test
0,0,1,1
1,9,1,1
2,13,1,1
3,20,0,0
4,33,1,0
...,...,...,...
2617,13088,1,1
2618,13091,0,0
2619,13093,0,0
2620,13094,0,0


In [36]:
baseline_pred

,id,y_test,y_pred
0,0,1,1
1,9,1,1
2,13,1,1
3,20,0,0
4,33,0,1
...,...,...,...
2617,13088,1,1
2618,13091,0,0
2619,13093,0,0
2620,13094,0,0


In [37]:
resnet_ft_pred

,index,id,y_test,y_pred
0,1267,0,1,0
1,1268,9,1,1
2,1269,13,1,0
3,0,20,0,0
4,1,33,0,0
...,...,...,...,...
2617,2620,13088,1,1
2618,1264,13091,0,0
2619,1265,13093,0,0
2620,1266,13094,0,0


In [38]:
b_corr_pred=[]
for index, row in baseline_pred.iterrows():
    if row['y_pred']== row['y_test']:
        b_corr_pred.append('1')
    else:
        b_corr_pred.append('0')


baseline_pred['prediction_correct']= b_corr_pred

r_corr_pred=[]
for index, row in resnet_ft_pred.iterrows():
    if row['y_pred']== row['y_test']:
        r_corr_pred.append('1')
    else:
        r_corr_pred.append('0')
        

resnet_ft_pred['prediction_correct']= r_corr_pred

v_corr_pred=[]
for index, row in vis_man_pred.iterrows():
    if row['y_pred']== row['y_test']:
        v_corr_pred.append('1')
    else:
        v_corr_pred.append('0')
        

vis_man_pred['prediction_correct']= v_corr_pred
df= pd.DataFrame(data= {'r': r_corr_pred, 'b': b_corr_pred, 'v': v_corr_pred})

### ResNet classifier against baseline


In [39]:
#define contingency table (false)
tab = pd.crosstab(df['r'], df['b'])

In [40]:
tab


b,0,1
r,,
0,9,167
1,170,2276


In [41]:
# Example of calculating the mcnemar test
from statsmodels.stats.contingency_tables import mcnemar
# define contingency table

# calculate mcnemar test
result = mcnemar(tab, exact=False)
# summarize the finding
print('statistic=%.3f, p-value=%.3f' % (result.statistic, result.pvalue))
# interpret the p-value
alpha = 0.05
if result.pvalue > alpha:
	print('Same proportions of errors (fail to reject H0)')
else:
	print('Different proportions of errors (reject H0)')

statistic=0.012, p-value=0.913
Same proportions of errors (fail to reject H0)


### Combined model against baseline


In [42]:
#define contingency table (false)
tab = pd.crosstab(df['v'], df['b'])

In [43]:
tab

b,0,1
v,,
0,115,50
1,64,2393


In [44]:
# Example of calculating the mcnemar test
from statsmodels.stats.contingency_tables import mcnemar
# define contingency table

# calculate mcnemar test
result = mcnemar(tab, exact=False)
# summarize the finding
print('statistic=%.3f, p-value=%.3f' % (result.statistic, result.pvalue))
# interpret the p-value
alpha = 0.05
if result.pvalue > alpha:
	print('Same proportions of errors (fail to reject H0)')
else:
	print('Different proportions of errors (reject H0)')

statistic=1.482, p-value=0.223
Same proportions of errors (fail to reject H0)


### Combined against ResNet

In [45]:
#define contingency table (false)
tab = pd.crosstab(df['v'], df['r'])

In [46]:
tab

r,0,1
v,,
0,9,156
1,167,2290


In [47]:
# Example of calculating the mcnemar test
from statsmodels.stats.contingency_tables import mcnemar
# define contingency table

# calculate mcnemar test
result = mcnemar(tab, exact=False)
# summarize the finding
print('statistic=%.3f, p-value=%.3f' % (result.statistic, result.pvalue))
# interpret the p-value
alpha = 0.05
if result.pvalue > alpha:
	print('Same proportions of errors (fail to reject H0)')
else:
	print('Different proportions of errors (reject H0)')

statistic=0.310, p-value=0.578
Same proportions of errors (fail to reject H0)
